In [15]:
import torch, torchvision
import torchinfo

DEVICE = torch.device("cuda:0") if torch.cuda.is_available() else "cpu"

In [20]:
model = torchvision.models.efficientnet_b3(parameters = {'weights': torchvision.models.EfficientNet_B3_Weights.DEFAULT}).to(DEVICE)

In [21]:
torchinfo.summary(model, (50, 3, 299, 299))

Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [50, 1000]                --
├─Sequential: 1-1                                       [50, 1536, 10, 10]        --
│    └─Conv2dNormActivation: 2-1                        [50, 40, 150, 150]        --
│    │    └─Conv2d: 3-1                                 [50, 40, 150, 150]        1,080
│    │    └─BatchNorm2d: 3-2                            [50, 40, 150, 150]        80
│    │    └─SiLU: 3-3                                   [50, 40, 150, 150]        --
│    └─Sequential: 2-2                                  [50, 24, 150, 150]        --
│    │    └─MBConv: 3-4                                 [50, 24, 150, 150]        2,298
│    │    └─MBConv: 3-5                                 [50, 24, 150, 150]        1,206
│    └─Sequential: 2-3                                  [50, 32, 75, 75]          --
│    │    └─MBConv: 3-6                            